# 7. Handling a MultiIndex
A MultiIndex or multi-level index is a cumbersome addition to a Pandas DataFrame that occasionally makes data easier to view, but often makes it much more difficult to manipulate. You usually encounter a MultiIndex after a `groupby` operation with multiple grouping columns or multiple aggregating columns. 

In [2]:
import pandas as pd
college = pd.read_csv('data/college.csv')
college.head()

,instnm,city,stabbr,hbcu,menonly,womenonly,relaffil,satvrmid,satmtmid,distanceonly,...,ugds_2mor,ugds_nra,ugds_unkn,pptug_ef,curroper,pctpell,pctfloan,ug25abv,md_earn_wne_p10,grad_debt_mdn_supp
0,Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,...,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
1,University of Alabama at Birmingham,Birmingham,AL,0.0,0.0,0.0,0,570.0,565.0,0.0,...,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5
2,Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,1.0,...,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370
3,University of Alabama in Huntsville,Huntsville,AL,0.0,0.0,0.0,0,595.0,590.0,0.0,...,0.0172,0.0332,0.0350,0.2146,1,0.3072,0.4596,0.2640,45500,24097
4,Alabama State University,Montgomery,AL,1.0,0.0,0.0,0,425.0,430.0,0.0,...,0.0098,0.0243,0.0137,0.0892,1,0.7347,0.7554,0.1270,26600,33118.5


Let's use a result similar to the one at the end of the previous notebook, except this time group by both state and religious affiliation.

In [3]:
df = college.groupby(['stabbr', 'relaffil']).agg({'satmtmid': ['min', 'max'],
                                                  'satvrmid': ['min', 'max'],
                                                  'ugds': 'mean'}).round(0)
df.head(10)

satmtmid        satvrmid           ugds
                     min    max      min    max    mean
stabbr relaffil                                        
AK     0             NaN    NaN      NaN    NaN  3509.0
       1           503.0  503.0    555.0  555.0   123.0
AL     0           420.0  590.0    424.0  595.0  3249.0
       1           400.0  560.0    420.0  565.0   980.0
AR     0           427.0  565.0    410.0  555.0  1794.0
       1           495.0  600.0    425.0  600.0   918.0
AS     0             NaN    NaN      NaN    NaN  1276.0
AZ     0           503.0  580.0    535.0  565.0  4364.0
       1           480.0  480.0    485.0  485.0   693.0
CA     0           445.0  785.0    435.0  765.0  3802.0

## A MultiIndex in both the index and columns
Both the rows and columns have a MultiIindex with two levels. Let's verify their types.

In [4]:
type(df.index)

pandas.core.indexes.multi.MultiIndex

In [ ]:
type(df.columns)

### Selection and further processing is difficult with a MultiIndex
There is no magic extra functionality that a MultiIndex possesses (outside of some trickery). They are harder to remember how to make selections from and more difficult to call other methods on. I suggest working with DataFrames that have a simpler, single-level index.

### Convert to a single level index - Rename the columns and reset the index
We can convert this DataFrame so that only single-level indexes rename. There is no direct way to rename columns of a DataFrame during a groupby (yes, something so simple is impossible with pandas), so we must overwrite them manually. Let's do that now.

In [5]:
df.columns = ['min satmtmid', 'max satmtmid', 'min satvrmid', 'max satvrmid', 'mean ugds']
df.head()

min satmtmid  max satmtmid  min satvrmid  max satvrmid  \
stabbr relaffil                                                           
AK     0                  NaN           NaN           NaN           NaN   
       1                503.0         503.0         555.0         555.0   
AL     0                420.0         590.0         424.0         595.0   
       1                400.0         560.0         420.0         565.0   
AR     0                427.0         565.0         410.0         555.0   

                 mean ugds  
stabbr relaffil             
AK     0            3509.0  
       1             123.0  
AL     0            3249.0  
       1             980.0  
AR     0            1794.0

From here, we can use the `reset_index` method to make each index level an actual column.

In [6]:
df_final = df.reset_index()
df_final.head()

,stabbr,relaffil,min satmtmid,max satmtmid,min satvrmid,max satvrmid,mean ugds
0,AK,0,NaN,NaN,NaN,NaN,3509.0
1,AK,1,503.0,503.0,555.0,555.0,123.0
2,AL,0,420.0,590.0,424.0,595.0,3249.0
3,AL,1,400.0,560.0,420.0,565.0,980.0
4,AR,0,427.0,565.0,410.0,555.0,1794.0
